# Importações e Configuração

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from zipfile import ZipFile
import kaggle

from transformer.utils.logger import get_logger

log = get_logger("landing.download_prepare")

DATASET_NAME = "usdot/flight-delays"
NUM_CHUNKS = 10

# Paths
stage_path = os.getenv("DATA_LAYER_STAGE_PATH", "/opt/airflow/data-layer/stage")
stage_path = Path(stage_path)
stage_path.mkdir(parents=True, exist_ok=True)

log.info(f"[Landing] Diretório configurado: {stage_path}")


# Definindo Funções de Download e de Divisão

In [ ]:
def download_kaggle_dataset(dataset_name: str, dest_path: Path) -> None:
    """
    Faz o download e a extração de um dataset do Kaggle, caso ainda não exista localmente.

    Args:
        dataset_name (str): Nome do dataset no Kaggle (ex: 'usdot/flight-delays').
        dest_path (Path): Caminho destino para os arquivos extraídos.

    Raises:
        Exception: Para erros de autenticação ou rede.
    """
    try:
        existing_files = os.listdir(dest_path)
        has_csv_files = any(file.endswith(".csv") for file in existing_files)

        if has_csv_files:
            log.info(f"[Landing] Arquivos já existem em '{dest_path}'. Pulando download.")
            return

        log.info(f"[Landing] Nenhum arquivo CSV encontrado. Baixando o dataset '{dataset_name}'.")
        kaggle.api.dataset_download_files(dataset_name, path=dest_path, unzip=True)
        log.info("[Landing] Download e extração concluídos com sucesso.")
    except Exception as e:
        log.exception(f"[Landing] Ocorreu um erro! Verifique se a API do Kaggle está configurada: {e}")
        log.exception(f"[Landing] Erro: {e}")
        raise


Nenhum arquivo CSV encontrado em 'Bronze'.
Baixando o dataset 'usdot/flight-delays' do Kaggle...
Dataset URL: https://www.kaggle.com/datasets/usdot/flight-delays

Download e extração concluídos com sucesso!


In [ ]:
def split_main_file(source_file: Path, num_chunks: int = NUM_CHUNKS) -> None:
    """
    Divide o arquivo flights.csv em múltiplos chunks de tamanho aproximadamente igual.

    Args:
        source_file (Path): Caminho do arquivo csv principal.
        num_chunks (int): Número de partes a gerar.

    Raises:
        FileNotFoundError: Se o arquivo não existir.
        ValueError: Se o arquivo estiver vazio.
        Exception: Para erros inesperados durante a divisão.
    """
    if not source_file.exists():
        raise FileNotFoundError(f"[Landing] Arquivo não encontrado: '{source_file}'")

    try:
        df = pd.read_csv(source_file)
        if df.empty:
            raise ValueError(f"O arquivo '{source_file}' está vazio.")

        log.info(f"[Landing] Dividindo '{source_file.name}' em {num_chunks} partes aproximadamente iguais...")
        chunks = np.array_split(df, num_chunks)

        for i, chunk_df in enumerate(chunks, start=1):
            chunk_name = f"{source_file.stem}_part_{i:02d}{source_file.suffix}"
            chunk_path = source_file.parent / chunk_name
            chunk_df.to_csv(chunk_path, index=False)
            log.info(f"[Landing] Chunk salvo: '{chunk_name}' ({len(chunk_df)} linhas)")

        log.info("[Landing] Divisão do arquivo concluída com sucesso.")
    except Exception as e:
        log.exception(f"[Landing] Erro ao dividir o arquivo: {e}")
        raise


# Executando as funções

In [ ]:
try:
    # Download
    download_kaggle_dataset(DATASET_NAME, stage_path)

    # Divisão
    flights_file = stage_path / "flights.csv"
    if flights_file.exists():
        split_main_file(flights_file, num_chunks=NUM_CHUNKS)
    else:
        log.warning(f"[Landing] Arquivo 'flights.csv' não encontrado. Verifique a pasta.")

except Exception as e:
    log.error(f"[Landing] Falha no processo de preparação: {e}")


# Verificação final e amostra dos dados

In [ ]:
log.info("[Landing] Verificando arquivos gerados na camada Bronze...")

files = sorted(f for f in os.listdir(stage_path) if f.endswith(".csv"))
for f in files:
    print(f"- {f}")

try:
    sample_file = next((stage_path / f for f in files if "flights_part" in f), None)
    if sample_file:
        log.info(f"[Landing] Lendo amostra do arquivo: {sample_file.name}")
        df_sample = pd.read_csv(sample_file)
        display(df_sample.head())
    else:
        log.warning("[Landing] Nenhum arquivo de chunk encontrado para amostra.")
except Exception as e:
    log.error(f"[Landing] Falha ao carregar amostra: {e}")



Arquivos disponíveis na camada Bronze:
- airlines.csv
- flights.csv
- docsCamadaBronze.pdf
- airports.csv

Lendo amostra do arquivo: 'airlines.csv'


,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
